# Asynchronous Programming
- 아래의 용어들과 혼용되지만, 주 실행흐름을 멈추지 않는 프로그래밍 방식을 말한다.

- Synchronous  : 통지 모델. application이 kernel을 체크한다.
- Asynchronous : 통지 모델. kernel이 application에게 결과를 통지한다.
- Blocking     : application이 Kernel의 응답을 대기한다. (결과가 나왔을 때 return)
- Not-blocking : application이 kernel의 응답을 대기하지 않는다. (결과에 상관 없이 return)

- Concurrency  : 실행 순서와 무관하게 동작
- Parallelism  : 많은 작업을 물리적으로 동시에 수행.

## 기본 라이브러리

In [1]:
const _ = require('./underscore-min.js');

undefined

__1.0 학생, 과목 정보 확인__
  - const, let
  - scope
  - sync / async

In [ ]:
{
    const fs = require('fs');
    const student = fs.readFileSync('./data/학생.csv', {encoding:'UTF-8'});
    const subject = fs.readFileSync('./data/교과목.csv', {encoding:'UTF-8'});
    console.log(student, subject);
}

__1.1 중복제거__
  - 함수만들기

In [ ]:
{
    const readFile = filename => {
        const fs = require('fs');
        return fs.readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    };
    console.log(readFile('학생.csv'), readFile('교과목.csv'));
}

__1.2 좀 더 줄여보기__
  - 화살표 함수의 생략

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    console.log(readFile('학생.csv'), readFile('교과목.csv'));
}

__2.0 JSON으로 파싱하기__
  - 함수 반환 하기
  - _.map
  - _.reduce

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const body    = csv.split('\n');
        const header  = body.shift().split(',');
        return _.map(body, data=>{
            return _.reduce(data.split(','), (m, v, i)=>{
                m[header[i]] = v;
                return m;
            }, {});
        });
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

__2.1 조금 줄여 보기__
  - 배열 비구분해 할당

In [ ]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body] = _.map(csv.split('\n'), d=>d.split(','));
        return _.map(body, data=>{
            return _.reduce(data, (m, v, i)=>{
                m[header[i]] = v;
                return m;
            }, {});
        });
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

__2.2 COMPOSE 사용해 보기__
  - 함수로 분해
  - _.compose

In [ ]:
{
    const readFile  = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const splitCr   = text => text.split('\n');
    const splitComma= text => text.split(',');
    const makeFunctionObjWithHeader = header => array => _.reduce(array, (m, v, i)=>(m[header[i]]=v,m),{});
    const csvToObj = csv => {
        const [header, ...body] = splitCr(csv);
        const funcArrayToObj = _.compose(
            _.compose(makeFunctionObjWithHeader, splitComma)(header),
            splitComma
        );
        return _.map(body, funcArrayToObj);
    };
    console.log(csvToObj(readFile('학생.csv')), csvToObj(readFile('교과목.csv')));
}

__2.3 줄여보기__

In [4]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const readCSV = _.compose(csvToObj, readFile);
    
    console.log(readCSV('학생.csv'), readCSV('교과목.csv'));
}

[
  { '학번': '100', '이름': '홍길동', '성별': '남', '나이': '19' },
  { '학번': '110', '이름': '허난설헌', '성별': '여', '나이': '20' },
  { '학번': '120', '이름': '전우치', '성별': '남', '나이': '30' },
  { '학번': '130', '이름': '심청', '성별': '여', '나이': '16' },
  { '학번': '140', '이름': '조조', '성별': '남', '나이': '23' },
  { '학번': '150', '이름': '관우', '성별': '남', '나이': '25' },
  { '학번': '160', '이름': '장비', '성별': '남', '나이': '19' }
] [
  { '교과번호': '100', '교과목명': '국어', '가중치': '1' },
  { '교과번호': '200', '교과목명': '영어', '가중치': '1' },
  { '교과번호': '300', '교과목명': '수학', '가중치': '1.1' },
  { '교과번호': '400', '교과목명': '체육', '가중치': '0.8' },
  { '교과번호': '500', '교과목명': '과학', '가중치': '0.9' },
  { '교과번호': '600', '교과목명': '음악', '가중치': '1.5' }
]


undefined

__3.0 시험 데이터 만들기__
  - for ... of

In [7]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const readCSV = _.compose(csvToObj, readFile);
    const makeScore = () => Math.floor(Math.random() * 41 + 60);
    
    const student = readCSV('학생.csv');
    const subject = readCSV('교과목.csv');
    
    for(const i of student) {
        for(const j of subject) {
            console.log(`${i['학번']},${j['교과번호']},${makeScore()}`);
        }
    }
}

100,100,73
100,200,73
100,300,77
100,400,76
100,500,98
100,600,67
110,100,100
110,200,75
110,300,91
110,400,63
110,500,73
110,600,86
120,100,93
120,200,96
120,300,95
120,400,96
120,500,98
120,600,91
130,100,87
130,200,93
130,300,98
130,400,82
130,500,75
130,600,77
140,100,65
140,200,76
140,300,79
140,400,84
140,500,86
140,600,76
150,100,72
150,200,64
150,300,60
150,400,82
150,500,61
150,600,67
160,100,83
160,200,83
160,300,91
160,400,98
160,500,89
160,600,79


undefined

__3.1 다른 순환 방법__
  - _.each

In [8]:
{
    const readFile = filename => require('fs').readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const readCSV = _.compose(csvToObj, readFile);
    const makeScore = () => Math.floor(Math.random() * 41 + 60);
    
    const student = readCSV('학생.csv');
    const subject = readCSV('교과목.csv');
    
    _.each(student, i => {
       _.each(subject, j => {
           console.log(`${i['학번']},${j['교과번호']},${makeScore()}`);
       });
    });
    console.log('end');
}

100,100,61
100,200,77
100,300,86
100,400,97
100,500,66
100,600,77
110,100,76
110,200,96
110,300,78
110,400,86
110,500,72
110,600,67
120,100,98
120,200,84
120,300,75
120,400,89
120,500,68
120,600,67
130,100,64
130,200,80
130,300,67
130,400,76
130,500,63
130,600,81
140,100,73
140,200,87
140,300,79
140,400,88
140,500,84
140,600,79
150,100,91
150,200,97
150,300,73
150,400,97
150,500,91
150,600,60
160,100,74
160,200,85
160,300,76
160,400,70
160,500,77
160,600,61
end


undefined

__4.0 시험 더미 파일 생성__

In [10]:
{
    const fs = require('fs');
    const readFile = filename => fs.readFileSync(`./data/${filename}`, {encoding:'UTF-8'});
    const saveFile = (filename, data) => fs.writeFileSync(`./data/${filename}`, data, {encoding:'UTF-8'});
    const csvToObj = csv => {
        const [header, ...body]= _.map(csv.split('\n'), v=>v.split(','));
        return _.map(body, data=>_.reduce(data, (m, v, i)=>(m[header[i]]=v,m),{}));
    };
    const readCSV = _.compose(csvToObj, readFile);
    const makeScore = () => Math.floor(Math.random() * 41 + 60);
    
    const student = readCSV('학생.csv');
    const subject = readCSV('교과목.csv');
    
    const buffer = [];
    buffer.push(`학번,교과번호,점수`);
    _.each(student, i => {
       _.each(subject, j => {
           buffer.push(`${i['학번']},${j['교과번호']},${makeScore()}`);
       });
    });
    saveFile('성적.csv', buffer.join('\n'));
    
    console.log(readCSV('성적.csv'));
}

[
  { '학번': '100', '교과번호': '100', '점수': '76' },
  { '학번': '100', '교과번호': '200', '점수': '86' },
  { '학번': '100', '교과번호': '300', '점수': '80' },
  { '학번': '100', '교과번호': '400', '점수': '74' },
  { '학번': '100', '교과번호': '500', '점수': '64' },
  { '학번': '100', '교과번호': '600', '점수': '89' },
  { '학번': '110', '교과번호': '100', '점수': '85' },
  { '학번': '110', '교과번호': '200', '점수': '88' },
  { '학번': '110', '교과번호': '300', '점수': '74' },
  { '학번': '110', '교과번호': '400', '점수': '61' },
  { '학번': '110', '교과번호': '500', '점수': '91' },
  { '학번': '110', '교과번호': '600', '점수': '75' },
  { '학번': '120', '교과번호': '100', '점수': '64' },
  { '학번': '120', '교과번호': '200', '점수': '95' },
  { '학번': '120', '교과번호': '300', '점수': '65' },
  { '학번': '120', '교과번호': '400', '점수': '68' },
  { '학번': '120', '교과번호': '500', '점수': '83' },
  { '학번': '120', '교과번호': '600', '점수': '77' },
  { '학번': '130', '교과번호': '100', '점수': '97' },
  { '학번': '130', '교과번호': '200', '점수': '78' },
  { '학번': '130', '교과번호': '300', '점수': '91' },
  { '학번': '130', '교과번호': '400', 

undefined